**The idea is:**

 - Feature reduction with PCA
 - Data transformation (log, hot encoding, nan)
 - Test different regression models

**Things found:**

- Applying log transformation really increases the accuracy.
- Using PCA with 36 components makes the learning and testing much (much much) faster.
- Removing columns with more than 1000 NaNs gives better result than applying "mean" to them.
- There are outliers. Instead of removing them, using Huber seems to provide a good result. Huber is a model robust to outliers.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.decomposition import PCA
from sklearn.preprocessing import Imputer
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.metrics import make_scorer
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.metrics import r2_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
#import tflearn
#import tensorflow as tf
import seaborn
import warnings
import subprocess
warnings.filterwarnings('ignore')

from subprocess import check_output
#subprocess.check_output(["ls","C:/Users/yuvalanavim/Desktop/test_regrission"])

In [2]:
data_path='/home/barlesh/Projects/Data_Mining_Project/data/'
train_file='train.csv'
test_file='test.csv'
train_file_new='train_new.csv'
test_file_new='test_new.csv'
result_path='results/'
result_file='res.csv'

## Data Load ##

I mix data and test to manipulate all the data just once. SalePrice is extracted to its own variable "labels". Finally, SalesPrice is remove from data.

In [3]:
train = pd.read_csv(data_path+train_file_new)
labels=train["SalePrice"]
test = pd.read_csv(data_path+test_file_new)
data = pd.concat([train,test],ignore_index=True)
data = data.drop("SalePrice", 1)
ids = test["Id"]

In [4]:
train.head()

,Unnamed: 0,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,1,60,RL,65.0,8450,Pave,0,Reg,Lvl,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,1,2,20,RL,80.0,9600,Pave,0,Reg,Lvl,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,2,3,60,RL,68.0,11250,Pave,0,IR1,Lvl,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,3,4,70,RL,60.0,9550,Pave,0,IR1,Lvl,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,4,5,60,RL,84.0,14260,Pave,0,IR1,Lvl,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [5]:
# Count the number of rows in train
train.shape[0]

1460

In [6]:
# Count the number of rows in total
data.shape[0]

2919

In [7]:
# Count the number of NaNs each column has.
nans=pd.isnull(data).sum()
nans[nans>0]

Series([], dtype: int64)

In [8]:
# Remove id and columns with more than a thousand missing values
#data=data.drop("Id", 1)
#data=data.drop("Alley", 1)
#data=data.drop("Fence", 1)
#data=data.drop("MiscFeature", 1)
#data=data.drop("PoolQC", 1)
#data=data.drop("FireplaceQu", 1)

In [9]:
# Count the column types
data.dtypes.value_counts()

object     44
int64      28
float64     9
dtype: int64

## Data Manipulation ##

- Apply hot encoding, convert categorical variable into dummy/indicator variables.
- Fill NaN with median for that column.
- Log transformation.
- Change -inf to 0.

In [10]:
all_columns = data.columns.values
non_categorical = ["LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1", 
                   "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", 
                   "2ndFlrSF", "LowQualFinSF", "GrLivArea", "GarageArea", 
                   "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", 
                   "ScreenPorch","PoolArea", "MiscVal"]

categorical = [value for value in all_columns if value not in non_categorical]

In [11]:
# One Hot Encoding and nan transformation
data = pd.get_dummies(data)

imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
data = imp.fit_transform(data)

# Log transformation
data = np.log1p(data)
labels = np.log1p(labels)

# Change -inf to 0 again
data[data==-np.inf]=0

## Feature reduction ##

There are many features, so I am going to use PCA to reduce them. The idea is to start with n_components = number of columns. Then select the number of components that add up to 1 variance_ratio.

In [12]:
pca = PCA(whiten=True)
pca.fit(data)
variance = pd.DataFrame(pca.explained_variance_ratio_)
np.cumsum(pca.explained_variance_ratio_)

array([ 0.19760458,  0.35926126,  0.46224055,  0.54787041,  0.61663246,
        0.67687481,  0.72556865,  0.76598861,  0.79627022,  0.82057729,
        0.84336107,  0.86289941,  0.87293647,  0.88216824,  0.89088168,
        0.89736578,  0.90258448,  0.9076625 ,  0.91230965,  0.91622932,
        0.91945689,  0.92229119,  0.92491808,  0.92746359,  0.92983136,
        0.93192809,  0.93398991,  0.9360133 ,  0.93797388,  0.93981181,
        0.94155563,  0.94324449,  0.94492936,  0.94650402,  0.94804956,
        0.94952207,  0.95096716,  0.952375  ,  0.95374932,  0.95502035,
        0.95626247,  0.95744185,  0.95859983,  0.95974299,  0.96084951,
        0.96189092,  0.96291092,  0.96391498,  0.96488338,  0.96580461,
        0.96671531,  0.9675925 ,  0.96844572,  0.96927717,  0.97009185,
        0.97086426,  0.97161593,  0.97234783,  0.97305419,  0.97375171,
        0.97443489,  0.97509531,  0.97574272,  0.97637504,  0.97700014,
        0.97759163,  0.97817923,  0.97874236,  0.97926415,  0.97

In [13]:
pca = PCA(n_components=50,whiten=True)
pca = pca.fit(data)
dataPCA = pca.transform(data)

In [14]:
print(pca)

PCA(copy=True, iterated_power='auto', n_components=50, random_state=None,
  svd_solver='auto', tol=0.0, whiten=True)


## Data Model Selection ##

Simple test to run multiple models against our data. First, with raw features. No PCA.

In [15]:
# Split traing and test
train = data[:1460]
test = data[1460:]

In [16]:
# R2 Score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error


def lets_try(train,labels):
    results={}
    def test_model(clf):
        
        cv = KFold(n_splits=5,shuffle=True,random_state=45)
        mse0 = make_scorer(mean_squared_error)
        mse0_val_score = cross_val_score(clf, train, labels, cv=cv,scoring=mse0)
        scores=[mse0_val_score.mean()]
        return scores

    clf = linear_model.LinearRegression()
    results["Linear"]=test_model(clf)
    
    clf = linear_model.Ridge()
    results["Ridge"]=test_model(clf)
    
    clf = linear_model.BayesianRidge()
    results["Bayesian Ridge"]=test_model(clf)
    
    clf = linear_model.HuberRegressor()
    results["Hubber"]=test_model(clf)
    
    clf = linear_model.Lasso(alpha=1e-4)
    results["Lasso"]=test_model(clf)
    
    clf = BaggingRegressor()
    results["Bagging"]=test_model(clf)
    
    clf = RandomForestRegressor()
    results["RandomForest"]=test_model(clf)
    
    clf = AdaBoostRegressor()
    results["AdaBoost"]=test_model(clf)
    
    clf = svm.SVR()
    results["SVM RBF"]=test_model(clf)
    
    clf = svm.SVR(kernel="linear")
    results["SVM Linear"]=test_model(clf)
    
    results = pd.DataFrame.from_dict(results,orient='index')
    results.columns=["mean_squared_error"] 
    results=results.sort(columns=["mean_squared_error"],ascending=False)
    results.plot(kind="bar",title="Model Scores")
    axes = plt.gca()
    axes.set_ylim([0.5,1])
    return results

lets_try(train,labels)

,mean_squared_error
Linear,1.649901e+18
AdaBoost,2.931139e-02
Bagging,2.398946e-02
RandomForest,2.373996e-02
SVM RBF,2.068017e-02
SVM Linear,1.917665e-02
Ridge,1.791998e-02
Bayesian Ridge,1.765186e-02
Lasso,1.702503e-02
Hubber,1.701929e-02


Now, let's try the same but using data with PCA applied.

In [17]:
# Split traing and test
train = dataPCA[:1460]
test = dataPCA[1460:]

lets_try(train,labels)

,mean_squared_error
AdaBoost,0.052711
Bagging,0.046489
RandomForest,0.044015
SVM RBF,0.032512
SVM Linear,0.024561
Linear,0.024433
Bayesian Ridge,0.024428
Ridge,0.024423
Lasso,0.024418
Hubber,0.024176


In [18]:
cv = KFold(n_splits=5,shuffle=True,random_state=45)

parameters = {'alpha': [1000,200,100,10],
              'epsilon' : [1.2,1.25,1.50,1.7],
              'tol' : [1e-10]}

clf = linear_model.HuberRegressor()
r2 = make_scorer(r2_score)
grid_obj = GridSearchCV(clf, parameters, cv=cv,scoring=r2)
grid_fit = grid_obj.fit(train, labels)
best_clf = grid_fit.best_estimator_ 

best_clf.fit(train,labels)

HuberRegressor(alpha=100, epsilon=1.7, fit_intercept=True, max_iter=100,
        tol=1e-10, warm_start=False)

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
mse1 = make_scorer(mean_squared_error)
scores = cross_val_score(clf, train, labels, cv=10, scoring=mse1)
print(scores)
print(np.mean(scores))

[ 0.02129803  0.01726948  0.02113003  0.03648716  0.03026765  0.01752579
  0.02183388  0.01863285  0.03526825  0.02023543]
0.0239948549804


In [21]:

predictions_huber = best_clf.predict(test)
predictions_huber = np.exp(predictions_huber)
sub = pd.DataFrame({
        "Id": ids,
        "SalePrice": predictions_huber
    })
sub.to_csv(data_path+result_path+"prices_huber1.csv", index=False)
#print(sub)

SVM tranning and predict

In [22]:
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, train, labels, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

In [23]:

print('Training on all data: ')
model_svm = svm.SVR(kernel="linear", C=3,)
model_svm.fit(train, labels)



cv_rmse_svm = rmse_cv(model_svm).mean()
print (cv_rmse_svm)






Training on all data: 
0.155331921673


In [24]:
predictions = model_svm.predict(test)
predictions_svm = np.exp(predictions)
sub = pd.DataFrame({
        "Id": ids,
        "SalePrice": predictions_svm
    })
sub.to_csv(data_path+result_path+"prices_svm1.csv", index=False)

In [21]:
mse1 = make_scorer(mean_squared_error)
scores = cross_val_score(clf, train, labels, cv=10, scoring=mse1)
print(scores)
print(np.mean(scores))

[ 0.02345859  0.01640437  0.02317852  0.03338032  0.029027    0.0226658
  0.02265506  0.02255614  0.03490317  0.02274871]
0.0250977686718


Simple Neural Network
---------------------

Now I am going to try a simple neural network, to see if i can improve the result.

In [ ]:
# Shape the labels
labels_nl = labels
labels_nl = labels_nl.reshape(-1,1)

In [ ]:
tf.reset_default_graph()
r2 = tflearn.R2()
net = tflearn.input_data(shape=[None, train.shape[1]])
net = tflearn.fully_connected(net, 30, activation='linear')
net = tflearn.fully_connected(net, 10, activation='linear')
net = tflearn.fully_connected(net, 1, activation='linear')
sgd = tflearn.SGD(learning_rate=0.1, lr_decay=0.01, decay_step=100)
net = tflearn.regression(net, optimizer=sgd,loss='mean_square',metric=r2)
model = tflearn.DNN(net)

In [ ]:
model.fit(train, labels_nl,show_metric=True,validation_set=0.2,shuffle=True,n_epoch=50)

In [ ]:
# Make predictions

predictions_huber = best_clf.predict(test)
predictions_DNN = model.predict(test)
predictions_huber = np.exp(predictions_huber)
predictions_DNN = np.exp(predictions_DNN)
predictions_DNN = predictions_DNN.reshape(-1,)

sub = pd.DataFrame({
        "Id": ids,
        "SalePrice": predictions_DNN
    })

sub.to_csv("prices_submission.csv", index=False)
#print(sub)